In [21]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

## Players
Imported player data and created table of usefull bits.

In [22]:
master = pd.read_csv('data/core/Master.csv', dtype=str)
master = master[['playerID', 'nameGiven', 'deathYear', 'finalGame']]
master.head()

,playerID,nameGiven,deathYear,finalGame
0,aardsda01,David Allan,NaN,2015-08-23
1,aaronha01,Henry Louis,NaN,1976-10-03
2,aaronto01,Tommie Lee,1984,1971-09-26
3,aasedo01,Donald William,NaN,1990-10-03
4,abadan01,Fausto Andres,NaN,2006-04-13


Deleted all players known to be dead from the list and deleted the deathYear column.

In [23]:
master = master[pd.isnull(master.deathYear)]
master = master.drop('deathYear', 1)
master.head()

,playerID,nameGiven,finalGame
0,aardsda01,David Allan,2015-08-23
1,aaronha01,Henry Louis,1976-10-03
3,aasedo01,Donald William,1990-10-03
4,abadan01,Fausto Andres,2006-04-13
5,abadfe01,Fernando Antonio,2015-10-03


### Data Cleanup - 2015 players
There is an abberation suggesting that 1300 people had their final game in 2015. We can assume that they did not all retire, since it's about 200 each year before that. I created a new variable to hold all players that retired before 2015 and deleted everyone with a value there. This allowed me to keep the players who had a NaN value for final game, as well as those who supposedly retired in 2015. Then I deleted the unneccessary columns.

In [24]:
master['is_inactive'] = master.finalGame[master.finalGame < '2015-01-01']
master = master[pd.notnull(master.is_inactive)]
master = master.drop(['is_inactive', 'finalGame'], 1)
master.head()

,playerID,nameGiven
1,aaronha01,Henry Louis
3,aasedo01,Donald William
4,abadan01,Fausto Andres
12,abbotgl01,William Glenn
13,abbotje01,Jeffrey William


Filtered player data by players without a final game (players that have not yet retired) and got rid of the column. Now I have a list of player ids and names for active players. There are 8114 living players in this database that had not retired as of 2014.

## Batting Data
Here I imported the stats for players and saved the valuable info.

In [69]:
batting = pd.read_csv('data/core/Batting.csv')
batting = batting[['playerID', 'G', 'AB', 'H', 'BB', 'HBP', 'SF']]
batting.head()

,playerID,G,AB,H,BB,HBP,SF
80632,abadan01,1,1.0,0.0,0.0,0.0,0.0
83290,abadan01,9,17.0,2.0,2.0,0.0,0.0
87314,abadan01,5,3.0,0.0,2.0,0.0,0.0


Each player had a stats for a particular season, so I got the mean of all of those stats by playerID.

In [26]:
batting = batting.groupby(['playerID']).mean()
batting.head()

,G,AB,H,BB,HBP,SF
playerID,,,,,,
aardsda01,36.777778,0.444444,0.000000,0.000000,0.000000,0.000000
aaronha01,143.391304,537.565217,163.956522,60.956522,1.391304,5.260870
aaronto01,62.428571,134.857143,30.857143,12.285714,0.000000,0.857143
aasedo01,34.461538,2.500000,0.000000,0.000000,0.000000,0.000000
abadan01,5.000000,7.000000,0.666667,1.333333,0.000000,0.000000


#### OBP
I replaced all of the NaN values with integer zeros so that I could still calculate OBP even if some information was missing. Then I calculated the On Base Percentage.

In [27]:
batting = batting.fillna(0)
batting['OBP'] = ((batting.H + batting.BB + batting.HBP)/(batting.AB + batting.BB + batting.HBP + batting.SF))

In [28]:
batting.sort_values('OBP').head()

,G,AB,H,BB,HBP,SF,OBP
playerID,,,,,,,
aardsda01,36.777778,0.444444,0.0,0.0,0.0,0.0,0.0
mickegl01,4.000000,2.000000,0.0,0.0,0.0,0.0,0.0
meyerda02,20.600000,0.400000,0.0,0.0,0.0,0.0,0.0
meyerbr01,11.333333,0.333333,0.0,0.0,0.0,0.0,0.0
merewar01,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0


I deleted all of the players with very small OBPs.

In [29]:
batting = batting[batting.OBP > 0.07]
batting['playerID'] = batting.index
batting.head()

,G,AB,H,BB,HBP,SF,OBP,playerID
playerID,,,,,,,,
aaronha01,143.391304,537.565217,163.956522,60.956522,1.391304,5.260870,0.373949,aaronha01
aaronto01,62.428571,134.857143,30.857143,12.285714,0.000000,0.857143,0.291506,aaronto01
abadan01,5.000000,7.000000,0.666667,1.333333,0.000000,0.000000,0.240000,abadan01
abadfe01,43.000000,1.333333,0.166667,0.000000,0.000000,0.000000,0.125000,abadfe01
abadijo01,6.000000,24.500000,5.500000,0.000000,0.000000,0.000000,0.224490,abadijo01


## Merge
I merged the batting stats with the payers given names. It's got 5884 players in it.

In [30]:
batting_p = pd.merge(batting, master)
batting_p = batting_p.sort_values('OBP', ascending=False)

Here I cut the data frame down to players with at least 17 At Bats per season. It's got 3972 players in it.

In [31]:
batting_p = batting_p[batting_p['AB'] > 17]
batting_p.head()

,G,AB,H,BB,HBP,SF,OBP,playerID,nameGiven
5713,13.000000,40.000000,17.000000,2.000000,0.000000,0.000000,0.452381,willigl01,Glenn David
2212,13.000000,24.000000,8.000000,5.000000,0.000000,0.000000,0.448276,hattijo01,John Duane
3913,7.000000,22.000000,7.000000,4.000000,1.000000,0.000000,0.444444,oliveda01,David Jacob
492,135.727273,447.590909,133.409091,116.272727,4.818182,4.136364,0.444295,bondsba01,Barry Lamar
5604,9.000000,28.000000,10.000000,4.000000,0.000000,0.000000,0.437500,watkito01,Thomas Gray


## Salaries
Here I imported the salary data.

In [35]:
salaries = pd.read_csv('data/core/Salaries.csv')
salaries.head()

,yearID,teamID,lgID,playerID,salary
0,1985,ATL,NL,barkele01,870000
1,1985,ATL,NL,bedrost01,550000
2,1985,ATL,NL,benedbr01,545000
3,1985,ATL,NL,campri01,633333
4,1985,ATL,NL,ceronri01,625000


In [36]:
salaries = salaries.groupby('playerID').mean()
salaries = salaries.drop('yearID', 1)
salaries['playerID'] = salaries.index
salaries.head()

,salary,playerID
playerID,,
aardsda01,1.322821e+06,aardsda01
aasedo01,5.750000e+05,aasedo01
abadan01,3.270000e+05,abadan01
abadfe01,6.291000e+05,abadfe01
abbotje01,2.462500e+05,abbotje01


## Moneyball merge
I brought together three data sets to create this DataFrame of players, stats, and average salaries.

In [67]:
money_ball = pd.merge(batting_p, salaries)
money_ball.head()

,G,AB,H,BB,HBP,SF,OBP,playerID,nameGiven,salary


## Appearances data (position)
I need a series of Basemen (1st, 2nd, 3rd), (Left, Center, Right) Fielders, Short Stops, Pitchers, and Catchers. My plan is to create a series for each position of player, merge each with the moneyball list, and find the best/cheapest players for each position.

In [80]:
appearances = pd.read_csv('data/core/Appearances.csv')
appearances.head()
x = appearances[appearances['playerID'] == 'abadan01']
x

,yearID,teamID,lgID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,...,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr
81491,2001,OAK,AL,abadan01,1.0,0.0,1,1.0,0,0,...,0,0,0,0,0,0,0,0.0,1.0,0.0
83411,2003,BOS,AL,abadan01,9.0,4.0,9,8.0,0,0,...,0,0,0,0,0,1,1,0.0,1.0,1.0
87541,2006,CIN,NL,abadan01,5.0,0.0,5,0.0,0,0,...,0,0,0,0,0,0,0,0.0,5.0,0.0


Changed G_ columns to represent the percentage of games played at each position.

In [81]:
appearances = appearances.groupby('playerID').mean()
appearances['G_p'] = appearances.G_p / appearances.G_all
appearances['G_c'] = appearances.G_c / appearances.G_all
appearances['G_1b'] = appearances.G_1b / appearances.G_all
appearances['G_2b'] = appearances.G_2b / appearances.G_all
appearances['G_3b'] = appearances.G_3b / appearances.G_all
appearances['G_ss'] = appearances.G_ss / appearances.G_all
appearances['G_lf'] = appearances.G_lf / appearances.G_all
appearances['G_cf'] = appearances.G_cf / appearances.G_all
appearances['G_rf'] = appearances.G_rf / appearances.G_all
appearances['G_of'] = appearances.G_of / appearances.G_all
appearances = appearances.drop(['yearID', 'G_all', 'GS', 'G_batting', 'G_defense', 'G_dh', 'G_ph', 'G_pr'], 1)
appearances.head()

,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of
playerID,,,,,,,,,,
aardsda01,1.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
aaronha01,0.0,0.0,0.063675,0.013038,0.002122,0.0,0.094906,0.088842,0.662220,0.836871
aaronto01,0.0,0.0,0.530892,0.016018,0.022883,0.0,0.311213,0.002288,0.004577,0.315789
aasedo01,1.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
abadan01,0.0,0.0,0.533333,0.000000,0.000000,0.0,0.000000,0.000000,0.066667,0.066667


In [111]:
first_base = appearances[appearances['G_1b'] > 0.6]
first_base = first_base[['G_1b']]
first_base['playerID'] = first_base.index
first_base = pd.merge(money_ball, first_base)
first_base.head()

,G,AB,H,BB,HBP,SF,OBP,playerID,nameGiven,salary,G_1b
0,132.176471,468.352941,148.176471,78.529412,3.352941,5.470588,0.413994,heltoto01,Todd Lynn,9.637647e+06,0.969292
1,143.333333,519.800000,154.266667,93.400000,8.533333,6.800000,0.407616,bagweje01,Jeffrey Robert,8.542268e+06,0.982326
2,75.636364,245.272727,65.727273,47.454545,6.727273,1.181818,0.398851,johnsni01,Nicholas Robert,2.680432e+06,0.808894
3,124.111111,421.777778,124.388889,70.833333,4.888889,5.333333,0.397967,olerujo01,John Garrett,4.549578e+06,0.918979
4,128.153846,428.000000,124.153846,74.230769,4.076923,5.153846,0.395849,hargrmi01,Dudley Michael,4.583330e+05,0.827131


In [110]:
second_base = appearances[appearances['G_2b'] > 0.6]
second_base = second_base[['G_2b']]
second_base['playerID'] = second_base.index
second_base = pd.merge(money_ball, second_base)
second_base.head()

,G,AB,H,BB,HBP,SF,OBP,playerID,nameGiven,salary,G_2b
0,136.000000,530.500000,153.250000,67.000000,11.583333,4.666667,0.377733,knoblch01,Edward Charles,3.842500e+06,0.846201
1,115.894737,422.000000,116.315789,65.421053,2.000000,3.368421,0.372851,randowi01,Willie Larry,8.168750e+05,0.977748
2,109.571429,397.142857,107.142857,61.000000,5.428571,2.714286,0.372243,verasqu01,Quilvio Alberto,1.537000e+06,0.953064
3,125.210526,477.526316,143.368421,54.315789,2.631579,5.105263,0.371245,alomaro01,Roberto,4.787738e+06,0.976040
4,118.117647,405.294118,107.823529,63.941176,5.058824,2.941176,0.370516,grichbo01,Robert Anthony,6.000000e+05,0.879980


In [109]:
third_base = appearances[appearances['G_3b'] > 0.6]
third_base = third_base[['G_3b']]
third_base['playerID'] = third_base.index
third_base = pd.merge(money_ball, third_base)
third_base.head()

,G,AB,H,BB,HBP,SF,OBP,playerID,nameGiven,salary,G_3b
0,135.555556,510.000000,167.222222,78.444444,1.277778,5.333333,0.414994,boggswa01,Wade Anthony,2.113288e+06,0.908197
1,131.526316,472.842105,143.473684,79.578947,0.947368,5.105263,0.401093,jonesch06,Larry Wayne,8.477607e+06,0.797919
2,133.555556,464.000000,124.111111,83.722222,4.388889,6.000000,0.380251,schmimi01,Michael Jack,2.161075e+06,0.920549
3,102.785714,377.000000,111.214286,47.785714,2.500000,2.928571,0.375394,seitzke01,Kevin Lee,6.671367e+05,0.730368
4,101.333333,351.916667,102.416667,45.250000,2.916667,4.000000,0.372654,muellbi02,William Richard,1.977944e+06,0.928454


In [108]:
short_stop = appearances[appearances['G_ss'] > 0.6]
short_stop = short_stop[['G_ss']]
short_stop['playerID'] = short_stop.index
short_stop = pd.merge(money_ball, short_stop)
short_stop.head()

,G,AB,H,BB,HBP,SF,OBP,playerID,nameGiven,salary,G_ss
0,137.350000,559.750000,173.250000,54.100000,8.500000,2.900000,0.377209,jeterde01,Derek Sanderson,1.392727e+07,0.973790
1,114.736842,417.736842,123.157895,49.421053,2.894737,3.526316,0.370527,larkiba01,Barry Louis,4.129868e+06,0.957339
2,95.600000,372.400000,116.466667,26.866667,3.933333,4.200000,0.361479,garcino01,Anthony Nomar,5.604814e+06,0.736402
3,93.214286,333.785714,95.071429,36.428571,1.571429,3.142857,0.354925,guillca01,Carlos Alfonso,5.308633e+06,0.655939
4,108.230769,347.846154,91.307692,43.769231,7.000000,3.153846,0.353628,blausje01,Jeffrey Michael,2.038577e+06,0.781805


In [107]:
left = appearances[appearances['G_lf'] > 0.6]
left = left[['G_lf']]
left['playerID'] = left.index
left = pd.merge(money_ball, left)
left.head()

,G,AB,H,BB,HBP,SF,OBP,playerID,nameGiven,salary,G_lf
0,135.727273,447.590909,133.409091,116.272727,4.818182,4.136364,0.444295,bondsba01,Barry Lamar,8.556606e+06,0.909243
1,106.241379,377.965517,105.344828,75.517241,3.379310,2.310345,0.401247,henderi01,Rickey Nelson Henley,2.310833e+06,0.786758
2,114.111111,425.444444,129.555556,57.666667,2.444444,4.888889,0.386724,greerru01,Thurman Clyde,3.147788e+06,0.802337
3,104.250000,369.666667,108.541667,55.416667,1.750000,3.166667,0.385368,raineti01,Timothy,2.027162e+06,0.785771
4,80.777778,259.777778,74.000000,40.555556,1.555556,2.000000,0.382084,danieka01,Kalvoski,8.259286e+05,0.874828


In [106]:
center = appearances[appearances['G_cf'] > 0.6]
center = center[['G_cf']]
center['playerID'] = center.index
center = pd.merge(money_ball, center)
center.head()

,G,AB,H,BB,HBP,SF,OBP,playerID,nameGiven,salary,G_cf
0,129.750000,491.812500,146.000000,66.812500,2.437500,4.000000,0.380931,willibe02,Bernabe,6.443750e+06,0.894509
1,122.944444,454.444444,131.944444,62.722222,2.111111,2.833333,0.376889,butlebr01,Brett Morgan,1.821023e+06,0.898780
2,105.842105,360.947368,102.578947,52.526316,2.578947,3.421053,0.375910,edmonji01,James Patrick,5.083700e+06,0.879165
3,98.307692,350.692308,99.846154,49.230769,2.384615,2.076923,0.374548,dykstle01,Leonard Kyle,2.809615e+06,0.949139
4,105.150000,406.000000,121.400000,47.250000,1.600000,3.250000,0.371644,loftoke01,Kenneth,3.563602e+06,0.945316


In [105]:
out = appearances[appearances['G_of'] > 0.6]
out = out[['G_of']]
out['playerID'] = out.index
out = pd.merge(money_ball, out)
out.head()

,G,AB,H,BB,HBP,SF,OBP,playerID,nameGiven,salary,G_of
0,135.727273,447.590909,133.409091,116.272727,4.818182,4.136364,0.444295,bondsba01,Barry Lamar,8.556606e+06,0.962492
1,109.619048,392.571429,122.571429,63.285714,5.190476,4.285714,0.410561,ramirma02,Manuel Aristides,1.088567e+07,0.843180
2,106.241379,377.965517,105.344828,75.517241,3.379310,2.310345,0.401247,henderi01,Rickey Nelson Henley,2.310833e+06,0.918858
3,110.444444,383.722222,120.000000,50.722222,7.666667,3.611111,0.400224,walkela01,Larry Kenneth Robert,7.357562e+06,0.907445
4,115.437500,407.937500,118.562500,73.937500,3.187500,4.375000,0.399821,gilesbr02,Brian Stephen,5.401578e+06,0.957228


In [103]:
pitcher = appearances[appearances['G_p'] > 0.6]
pitcher = pitcher[['G_p']]
pitcher['playerID'] = pitcher.index
pitcher = pd.merge(money_ball, pitcher)
pitcher.head()

,G,AB,H,BB,HBP,SF,OBP,playerID,nameGiven,salary,G_p
0,26.142857,29.285714,8.285714,1.142857,0.142857,0.285714,0.310185,owingmi01,Micah Burton,4.405000e+05,0.754098
1,20.000000,17.250000,4.750000,0.750000,0.000000,0.000000,0.305556,chatwty01,Tyler Cole,4.910000e+05,0.875000
2,28.000000,45.312500,11.125000,2.937500,0.312500,0.312500,0.294118,hamptmi01,Michael William,7.326486e+06,0.935268
3,20.900000,29.166667,7.500000,1.500000,0.000000,0.000000,0.293478,watsoal01,Allen Kenneth,8.784000e+05,0.985646
4,24.866667,34.571429,7.142857,4.000000,0.000000,0.000000,0.288889,stottto01,Todd Vernon,3.825607e+06,0.997319


In [104]:
catcher = appearances[appearances['G_c'] > 0.6]
catcher = catcher[['G_c']]
catcher['playerID'] = catcher.index
catcher = pd.merge(money_ball, catcher)
catcher.head()

,G,AB,H,BB,HBP,SF,OBP,playerID,nameGiven,salary,G_c
0,31.000000,82.800000,23.400000,13.600000,1.000000,1.000000,0.386179,melusmi01,Mitchell Wade,2.550000e+05,0.800000
1,41.000000,115.000000,35.000000,11.000000,3.000000,1.000000,0.376923,henlebo01,Robert Clifton,1.950000e+05,0.853659
2,106.222222,383.944444,118.166667,42.166667,1.666667,2.500000,0.376501,piazzmi01,Michael Joseph,8.011733e+06,0.854603
3,107.588235,358.352941,97.882353,55.058824,4.352941,2.764706,0.374038,posadjo01,Jorge Rafael,7.830567e+06,0.860580
4,89.400000,282.000000,73.900000,43.500000,4.400000,2.800000,0.366096,hoilech01,Christopher Allen,1.792722e+06,0.916107


In [112]:
min(catcher['salary'])

60000.0

In [113]:
max(catcher['salary'])

8011733.4666666668